In [33]:
# import libraries you might need
import pandas as pd;
import numpy as np;
import math;
import re; # regex
import os;

# Format new campus and district entity names

In `scuole-data`, we have an `entities.csv` file at the district and county level that contains details used to create District and County models. Each year, we need to update it because districts and schools can get renamed/removed/added.

Change the year variable to the latest school year you are adding the data about.

We take the campus and district names from these files and format them. The formatted names are in the column `XX_CLEAN`

In [23]:
year = '2021-2022'
path = './tapr/' + year + "/"

In [46]:
# initial format of the reference files

# CAMPUS
level = 'campus'

# set the file path
file = path + level + '/reference.csv'
new_file = path + level + '/entities_' + level + '.csv'

# edit the reference file
reference_edited_rating = pd.read_csv(file)

# fill in zeros for ID
reference_edited_rating['CAMPUS'] = reference_edited_rating['CAMPUS'].apply(lambda x: str(x).replace("'", '').zfill(9))
reference_edited_rating['DISTRICT'] = reference_edited_rating['DISTRICT'].apply(lambda x: str(x).replace("'", '').zfill(6))
reference_edited_rating['COUNTY'] = reference_edited_rating['COUNTY'].apply(lambda x: str(x).replace("'", ''))
reference_edited_rating['REGION'] = reference_edited_rating['REGION'].apply(lambda x: str(x).replace("'", ''))
reference_edited_rating['GRDSPAN'] = reference_edited_rating['GRDSPAN'].apply(lambda x: str(x).replace("'", ''))

reference_edited_rating.to_csv(new_file, index=False)

# DISTRICT
level = 'district'
# set the file path
file = path + level + '/reference.csv'
new_file = path + level + '/entities_' + level + '.csv'

# edit the reference file
reference_edited_rating = pd.read_csv(file)

# fill in zeros for ID
reference_edited_rating['DISTRICT'] = reference_edited_rating['DISTRICT'].apply(lambda x: str(x).replace("'", '').zfill(6))
reference_edited_rating['COUNTY'] = reference_edited_rating['COUNTY'].apply(lambda x: str(x).replace("'", ''))
reference_edited_rating['REGION'] = reference_edited_rating['REGION'].apply(lambda x: str(x).replace("'", ''))

reference_edited_rating.to_csv(new_file, index=False)

In [42]:
# If the name is MCKINLEY
# We want to make sure McKinley is shown
def uppercase_mc(name_portion):
    mc_re = r'(?i)\b(?P<mc>ma?c)(?!hin)(?P<first_letter>\w)\w+'
    matches = re.search(mc_re, name_portion)

    if matches:
        mc = matches.group('mc')
        first_letter = matches.group('first_letter')
        return re.sub(mc + first_letter, mc.title() + first_letter.upper(), name_portion)
    else:
        return name_portion

In [47]:
# DISTRICT ENTITIES
level = 'district'
file = path + level + '/entities_' + level + '.csv'
new_file = path + level + '/entities.csv'

# read district files
entities_district = pd.read_csv(file)

# add the modified name in column 'DISTNAME_CLEAN'
# correct district names with 'Mc' in them (i.e. Mckinley -> McKinley)
entities_district['DISTNAME_CLEAN'] = entities_district['DISTNAME'].apply(lambda x: re.sub('\s+Isd*(?!.)', ' ISD', x.title()))
entities_district['DISTNAME_CLEAN'] = entities_district['DISTNAME_CLEAN'].apply(lambda x: re.sub('\s+Cisd*(?!.)', ' CISD', x))
entities_district['DISTNAME_CLEAN'] = entities_district['DISTNAME_CLEAN'].apply(lambda x: re.sub('\s+Csd*(?!.)', ' CSD', x))
entities_district['DISTNAME_CLEAN'] = entities_district['DISTNAME_CLEAN'].apply(lambda x: re.sub('\s+Llc*(?!.)', ' LLC', x))
entities_district['DISTNAME_CLEAN'] = entities_district['DISTNAME_CLEAN'].apply(lambda x: re.sub("'S", "'s", x)) # needs to come after title casing
entities_district['DISTNAME_CLEAN'] = entities_district['DISTNAME_CLEAN'].apply(lambda x: uppercase_mc(x))
                         
# NEED TO FORMAT DISTRICT, COUNTY AND REGION
entities_district['DISTRICT'] = entities_district['DISTRICT'].astype(int)
entities_district['COUNTY'] = entities_district['COUNTY'].astype(int)
entities_district['REGION'] = entities_district['REGION'].astype(int)

entities_district = entities_district.drop(columns=['D_RATING'])

# write to a CSV
entities_district.to_csv(new_file, index=False)

# delete old file
os.remove(file)

In [48]:
# CAMPUS ENTITIES
level = 'campus'
file = path + level + '/entities_' + level + '.csv'
new_file = path + level + '/entities.csv'

# read campus files
entities_campus = pd.read_csv(file)

# for campuses with no modified name, add the modified name in column 'CAMPNAME_CLEAN'
# do a bunch of replacements for abbreviated campus names
entities_campus['CAMPNAME_CLEAN'] = entities_campus['CAMPNAME'].apply(lambda x: re.sub('\s+H S*(?!.)', ' High School', x))
entities_campus['CAMPNAME_CLEAN'] = entities_campus['CAMPNAME_CLEAN'].apply(lambda x: re.sub('\s+J H*(?!.)', ' Junior High', x))
entities_campus['CAMPNAME_CLEAN'] = entities_campus['CAMPNAME_CLEAN'].apply(lambda x: re.sub('\s+EL*(?!.)', ' Elementary School', x))
entities_campus['CAMPNAME_CLEAN'] = entities_campus['CAMPNAME_CLEAN'].apply(lambda x: re.sub('\s+PRI*(?!.)', ' Primary School', x))
entities_campus['CAMPNAME_CLEAN'] = entities_campus['CAMPNAME_CLEAN'].apply(lambda x: re.sub('\s+LRN CTR*(?!.)', ' Learning Center', x))
entities_campus['CAMPNAME_CLEAN'] = entities_campus['CAMPNAME_CLEAN'].apply(lambda x: re.sub('\s+DETENT CTR*(?!.)', ' Detention Center', x))
entities_campus['CAMPNAME_CLEAN'] = entities_campus['CAMPNAME_CLEAN'].apply(lambda x: re.sub('\s+EDUC CTR*(?!.)', ' Education Center', x))
entities_campus['CAMPNAME_CLEAN'] = entities_campus['CAMPNAME_CLEAN'].apply(lambda x: re.sub('\s+MIDDLE*(?!.)', ' Middle School', x))
entities_campus['CAMPNAME_CLEAN'] = entities_campus['CAMPNAME_CLEAN'].apply(lambda x: re.sub('\s+INT*(?!.)', ' Intermediate School', x))
entities_campus['CAMPNAME_CLEAN'] = entities_campus['CAMPNAME_CLEAN'].apply(lambda x: re.sub('\s+AEC*(?!.)', ' Alternative Education Center', x))
entities_campus['CAMPNAME_CLEAN'] = entities_campus['CAMPNAME_CLEAN'].apply(lambda x: re.sub('\s+JUSTICE C*(?!.)', ' Justice Center', x))
entities_campus['CAMPNAME_CLEAN'] = entities_campus['CAMPNAME_CLEAN'].apply(lambda x: re.sub('\s+CTR*(?!.)', ' Center', x))
entities_campus['CAMPNAME_CLEAN'] = entities_campus['CAMPNAME_CLEAN'].apply(lambda x: re.sub('\s+CEN*(?!.)', ' Center', x))

# title case everything
# excluse JJAEP, DAEP, PK-8, J J A E P
entities_campus['CAMPNAME_CLEAN'] = entities_campus['CAMPNAME_CLEAN'].apply(lambda x: x.title().replace('Jjaep', 'JJAEP').replace('Daep', 'DAEP').replace('Pk-8', 'PK-8').replace('Pk - 8', 'PK-8').replace('J J A E P', 'JJAEP'))

# replace 'S with 's
# needs to come after title casing
entities_campus['CAMPNAME_CLEAN'] = entities_campus['CAMPNAME_CLEAN'].apply(lambda x: re.sub("'S", "'s", x))

# correct campus names with 'Mc' in them (i.e. Mckinley -> McKinley)
entities_campus['CAMPNAME_CLEAN'] = entities_campus['CAMPNAME_CLEAN'].apply(lambda x: uppercase_mc(x))

# NEED TO FORMAT THE COUNTY AND REGION
entities_campus['COUNTY'] = entities_campus['COUNTY'].astype(int)
entities_campus['DISTRICT'] = entities_campus['DISTRICT'].astype(int)

entities_campus = entities_campus.drop(columns=['C_RATING'])

# write to a CSV
entities_campus.to_csv(new_file, index=False)

# delete old file
os.remove(file)